In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import statsmodels.formula.api as smf

import statsmodels.api as sm
import statsmodels.formula.api as smf

## Load the data

In [ ]:
td = pd.read_csv('T1_trainingData.csv')
manID = pd.read_excel('manID.xlsx')
womanID = pd.read_excel('womanID.xlsx')

# Create a DataFrame for gender mapping
manID['gender'] = 0
womanID['gender'] = 1

# Combine the manID and womanID into a single DataFrame
gender_mapping = pd.concat([manID, womanID], ignore_index=True)

# Ensure that the gender mapping DataFrame only contains ID and gender columns
gender_mapping = gender_mapping[['ID', 'gender']]

# Merge the gender mapping with td based on the ID
gender_mapping['ID'] = gender_mapping['ID'].str.replace('_', '')
gender_mapping['ID'] = pd.to_numeric(gender_mapping['ID'], errors='coerce')
gender_mapping
td = td.merge(gender_mapping, on='ID', how='left')

# For any IDs that were not found in the gender mapping, fill with 'Unknown'
td['gender'] = td['gender'].fillna('Unknown')


## Mixed learning Model

In [ ]:
#NORMALIZATION WITH MINMAX
# Initialize the MinMaxScaler
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

# Normalize the features 'F1' and 'F2'
td[['F1', 'HNR' , 'Shimmer' , 'F2' , 'Jitter']] = scaler_feature.fit_transform(td[['F1', 'HNR' , 'Shimmer' , 'F2' , 'Jitter']])
td[['UPDRS3_vf']] = scaler_target.fit_transform(td[['UPDRS3_vf']])


# Fit the model  + Jitter + F2
model = smf.mixedlm("UPDRS3_vf ~ Shimmer + Jitter + HNR + F1 + F2", td, groups=td["ID"])
fitted_model = model.fit()
print(fitted_model.summary())
print(fitted_model.conf_int())

# Predictions and actual values
y_true = td['UPDRS3_vf']
y_pred = fitted_model.fittedvalues

mae = mean_absolute_error(y_true, y_pred)
print("Mean Absolute Error (MAE):", mae)

epsilon = 1e-10
# Relative Absolute Error Calculation
relative_absolute_error = np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + epsilon))

# Cross-Validation setup with GroupKFold (5 folds)
group_kfold = GroupKFold(n_splits=5)
mse_cv = []
relative_absolute_error_cv = []
maerror = []

for train_index, test_index in group_kfold.split(td, groups=td['ID']):
    train_data, test_data = td.iloc[train_index], td.iloc[test_index]
    
    model_cv = smf.mixedlm("UPDRS3_vf ~ Shimmer + Jitter + HNR + F1 + F2 ", train_data, groups=train_data["ID"])
    fitted_model_cv = model_cv.fit()
    y_true_cv = test_data['UPDRS3_vf']
    y_pred_cv = fitted_model_cv.predict(test_data)
    
    mse_cv.append(mean_squared_error(y_true_cv, y_pred_cv))
    maerror.append(mean_absolute_error(y_true_cv, y_pred_cv))
    relative_absolute_error_cv.append(np.mean(np.abs(y_true_cv - y_pred_cv) / (np.abs(y_true_cv) + epsilon)))

# Mean of cross-validated metrics
mse_cv_mean = np.mean(mse_cv)
relative_absolute_error_cv_mean = np.mean(relative_absolute_error_cv)
maerror_normalized  = np.mean(maerror)

maerror_denormalized = maerror_normalized * (scaler_target.data_max_[0] - scaler_target.data_min_[0]) + scaler_target.data_min_[0]
#maerror_denormalized = scaler_target.inverse_transform([[maerror_normalized]])[0, 0]
# P-value analysis
p_values = fitted_model.pvalues

# Collecting results
results = {
    "Relative Absolute Error": relative_absolute_error,
    "Cross-validated MSE (Mean)": mse_cv_mean,
    "Cross-validated Relative Absolute Error (Mean)": relative_absolute_error_cv_mean,
    "P-values": p_values,
   "Mean Absolute Error (MAE) (Normalized)": maerror_normalized,
    "Mean Absolute Error (MAE) (Denormalized)": maerror_denormalized
}

print(results)
residuals = 0
import matplotlib.pyplot as plt

residuals = fitted_model.resid
plt.hist(residuals, bins=30)
plt.title('Residuals Histogram')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()